# Executive Order Tagging

Use the clusters to figure out tags.

In [1]:
import json
import pandas
from typing import List
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field

## Combine Datasets

Combine some datasets so we can understand which presidents and parties are in each cluster.

In [33]:
clusters = pandas.read_csv('data/executive_orders/clusters.csv')
clusters['name'] = clusters['path'].apply(lambda x: x.split('/')[-1].split('.')[0])

sentiment = pandas.read_csv('data/executive_orders/sentiment.csv')
presidents = pandas.read_csv('data/executive_orders/presidents.csv')

# add president to the clusters
combined = clusters.merge(sentiment, on='name')
# join with president
combined = combined.merge(presidents, on='president')

# save
combined.to_csv('data/executive_orders/combined.csv', index=False)

In [2]:
combined = pandas.read_csv('data/executive_orders/combined.csv')

# group by cluster and create a column of array of unique presidents and parties
grouped = combined.groupby('cluster').agg({
    'president': lambda x: list(set(x)),
    'party': lambda x: list(set(x))
}).reset_index()
grouped

,cluster,president,party
0,0,"[Chester A. Arthur, Richard Nixon, Andrew John...","[Democratic, Republican, Whig]"
1,1,"[Richard Nixon, Rutherford B. Hayes, Harry S T...","[Democratic, Republican]"
2,2,"[Chester A. Arthur, Donald J. Trump (2nd Term)...","[Democratic, Republican]"
3,3,"[George W. Bush, Donald J. Trump (2nd Term), U...","[Democratic, Republican]"
4,4,"[Harry S Truman, George W. Bush, Gerald R. For...","[Democratic, Republican]"
5,5,"[Richard Nixon, Donald J. Trump (1st Term), An...","[Democratic, Republican]"
6,6,"[Harry S Truman, Theodore Roosevelt, Gerald R....","[Democratic, Republican]"
7,7,"[Harry S Truman, George W. Bush, Gerald R. For...","[Democratic, Republican]"
8,8,"[Harry S Truman, Chester A. Arthur, Woodrow Wi...","[Democratic, Republican]"
9,9,"[Chester A. Arthur, Richard Nixon, Andrew John...","[Whig, No Party, Democratic-Republican, Democr..."


## Build Tags

In [ ]:
gemini = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.1,
    max_retries=3
)

class Tags(BaseModel):
    analysis: str = Field(description='Analysis of the texts.')
    tags: List[str] = Field(description='Tags assigned to the cluster..')
    title: str = Field(description='Title of the cluster.')

def build_tags_pipeline(model, n_texts: int):
    prompt = [
        'Your task is to understand why the given documents were assigned to the same cluster of documents of the same type.',
        '- First analyze the texts for common themes, topics, and sentiment.',
        '- Then assign tags that would be unique to this cluster based on your analysis.',
        '- Finally give the cluster a concise title based on the analysis and derived tags to distinguish it from the other clusters.',
        '- Do not use tags or titles that describe the document type, such as "Executive Order" or "Presidential Orders".',
        '- Avoid generic phrases like "Governmental Policies" or "Governmental Regulations", instead describe what is in common across all documents.'
    ]
    for i in range(n_texts):
        prompt.extend([
            f'# Document {i+1}',
            f'{{document_{i}}}',
        ])

    return ChatPromptTemplate.from_template(
        '\n'.join(prompt)
    ) | model.with_structured_output(Tags, include_raw=True)

def sample_in_cluster(cluster: int, n_texts: int):
    paths = combined[combined['cluster'] == cluster]['path'].sample(n_texts)

    documents = []
    for p in paths:
        with open('data/executive_orders/raw/' + p) as f:
            eo = json.load(f)
            documents.append('\n'.join(eo["content"]))

    return documents

In [50]:
responses = []
for i in range(15):
    response = build_tags_pipeline(gemini, 4)\
        .invoke({f'document_{d_i}': d for d_i, d in enumerate(sample_in_cluster(2, 4))})
    responses.append(response)

    print(f'{str(i)}: {response['parsed'].title}')

0: Promoting Competition, Security, and Innovation
1: Government Land Control and Economic Measures
2: Federal Personnel and Homeland Security Organization
3: Immigration, Border Security, and Visa Policies
4: National Resource Management and Security Directives
5: Governmental Authority and Resource Allocation for Public Works
6: Governmental Regulations on Personnel, Properties, and International Affairs
7: National Emergency Measures and Government Preparedness
8: Governmental Regulations on Land, Labor, Family, and Maritime Affairs
9: AI, International Security, Military Service, and Water Management
10: Advancing Public Health, Education, and Economic Equity
11: Governmental Regulations on Public Health, Economic Sanctions, and Infrastructure
12: Governmental Directives on Employment, Security, and Conduct
13: Federal Employee Policies and Hispanic Community Initiatives
14: Presidential Actions on Infrastructure, Emergencies, Equity, and Labor


In [51]:
responses = []
for i in range(15):
    response = build_tags_pipeline(gemini, 4)\
        .invoke({f'document_{d_i}': d for d_i, d in enumerate(sample_in_cluster(2, 4))})
    responses.append(response)

    print(f'{str(i)}: {response['parsed'].title}')

0: Updates to Military Courts-Martial Procedures
1: Emergency Response and Executive Action
2: Land Management, Property Rights, and Infrastructure Initiatives
3: Protecting National Security, Promoting Human Rights, and Regulating Military Service
4: Government Ethics and Labor Dispute Resolution
5: Securing National Interests: Resource Allocation, Supply Chains, and Investment Oversight
6: LGBTQI+ Rights, Peace Corps, North Korea Sanctions, and Energy Infrastructure
7: Amendments to Military, Health, and Civilian Regulations
8: Federal Directives on Safety, Health, and Global Security
9: Governmental Regulations and Industry Oversight
10: Governmental Regulations and Policy Implementation
11: Land Transfer, Security, Arctic, and Wage Policies
12: Governmental Frameworks for National Security, Resource Management, and Public Health
13: Federal Agency Authority on Environment, Military Pay, Trade, and Aid
14: Federal Authority and Policy Directives


In [52]:
responses = []
for i in range(15):
    response = build_tags_pipeline(gemini, 4)\
        .invoke({f'document_{d_i}': d for d_i, d in enumerate(sample_in_cluster(2, 4))})
    responses.append(response)

    print(f'{str(i)}: {response['parsed'].title}')

0: Papago Reservation, Seafood Regulations, Cultural Exchange, and Visa Fees
1: Presidential Delegations of Authority
2: Government Employee Travel, Climate Resilience, and Cybersecurity Policies
3: Governmental Authority and Regulatory Procedures
4: Safeguarding National Security: Data, Emergencies, Cyber Threats, and Atrocity Prevention
5: Sector-Specific Governmental Regulations and Policy Implementation
6: Regulatory Amendments: Civil Service, North Korea, and Selective Service
7: Governmental Actions on Child Welfare, Land Management, Property Rights, and Science Recognition
8: Federal Directives on Land, Justice, Equality, and Resources
9: Native American Education, Federal Background Checks, Military Service & Alaska Railroad Labor Regulations
10: Governmental Regulations on Health, Military Conduct, Land Management, and Federal Salaries
11: Presidential Directives on National Issues
12: AI Advancement, Employee Leave, Non-Discrimination, and Order Revocation
13: Environmental P

In [53]:
responses = []
for i in range(15):
    response = build_tags_pipeline(gemini, 4)\
        .invoke({f'document_{d_i}': d for d_i, d in enumerate(sample_in_cluster(2, 4))})
    responses.append(response)

    print(f'{str(i)}: {response['parsed'].title}')

0: Governmental Regulations: Passports, Ecosystems, Military Service, and West Bank Security
1: Presidential Directives on Policy and Administration
2: Governmental Orders on Land, Healthcare, Security, and Military Service
3: WMD Control, Wartime Food Supply, Labor Standards, and Employee Leave
4: Combating Societal Issues and Promoting Equity through Governmental Action
5: Regulations on Exports, Water Quality, Exhibitions, and Alaskan Land Use
6: Infrastructure, Mental Health, Wartime Production, and Voting Rights Initiatives
7: Governmental Actions on Military, Social, Economic, and Environmental Issues
8: Governmental Actions: Amendments, Labor, Public Health, and Land Use
9: Detention Review, Land Withdrawal, Healthcare & Military Service Regulations
10: Land Management and Territorial Control
11: Governmental Regulations on Education, Military Justice, International Relations, and Civil Service
12: Executive Actions Addressing National Concerns
13: Military Justice, Employee Ben